# Technical analysis
This file calculates and plots Bollinger and Ichimoku
(Also calculates daily and cumulative return)

The output of this file is a list of the top 10 stocks (by cumulative return) of each sector and their plot with Ichimoku clouds in a subfolder.

## Load relevant libraries

In [24]:
# Data management
import numpy as np
import pandas as pd

# For plotting 
import matplotlib.pyplot as plt
import matplotlib.dates as mdates # for styling dates

from plotly.offline import plot 
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import cufflinks as cf # an addon to plotly
# Use plotly locally
cf.go_offline()

# For managing files
import time
# import datetime as dt
import os
from os import listdir
from os.path import isfile, join

# For managing warnings
import warnings
warnings.simplefilter("ignore")

## Constants

In [25]:
# Working directory
# plots_path = "D:\\webScrapping\\derekBanasTutorial\\"
# downloaded stocks path
stocks_path = "D:\\webScrapping\\derekBanasTutorial\\wilshire_stocks\\"
# plots folder 
plots_path = "D:\\webScrapping\\derekBanasTutorial\\wilshire_top\\"

# Start & end dates default
# S_DATE = '2017-06-19'
# E_DATE = '2018-06-19'
# S_DATE_DT = pd.to_datetime(S_DATE)
# E_DATE_DT = pd.to_datetime(E_DATE)

# risk_free_rate = 0.0125 # Approximate 10 year bond rate
# a rate to beat?

## Get tickers from downloaded stock files

In [26]:
files = [x for x in listdir(stocks_path) if isfile(join(stocks_path, x))] 
tickers = [os.path.splitext(x)[0] for x in files] # split file extension 
tickers.sort() # although they should be sorted by windows
print("There are ", len(tickers), " tickers")

There are  2950  tickers


# FUNCTIONS
***

## Get dataframe from csv files function

In [27]:
def get_stock_df_from_csv(ticker):
    try:
        df = pd.read_csv(stocks_path + ticker + '.csv', index_col=0)
        
        # Check if df has duplicate indexes
        # Sometimes yahoo data comes duplicated
        if not df.index.is_unique:
            df = df.loc[~df.index.duplicated(), :] # ~ is the "invert" or "complement" operation
            # is the bitwise complement operator in python which essentially calculates -x - 1
            
    except FileNotFoundError:
        print("File doesn't exist!")
    else:
        return df

## Calculate daily returns and cumulative daily returns from stocks dataframes

In [28]:
def add_daily_return_to_df(df):
    '''
    Shifting the df by 1 means looking the previous day's value
    This is a rate of return (percentage)
    (close + previous close)/ previous close
    '''
    df['daily_return'] = (df['Close'] / df['Close'].shift(1)) - 1
    return df

def add_cumulative_return_to_df(df):
    '''
    The aggregate effect of price change, compounded (daily in my case)
    Cumprod returns cumulative product, i.e. it compounds
    '''
    df['cum_return'] = (1 + df['daily_return']).cumprod()
    return df

## Add Bollinger bands data
Basically a moving average line and a band showing the standard deviation

In [29]:
def add_bollinger_bands(df):
    df['middle_band'] = df['Close'].rolling(window=20).mean() # Consider using 22D to work in days
    df['upper_band'] = df['middle_band'] + 1.96 * df['Close'].rolling(window=20).std()
    df['lower_band'] = df['middle_band'] - 1.96 * df['Close'].rolling(window=20).std()
    return df

## Add ichimoku data to dataframe
My personal view: This is just averaging the future using the past. Averaging! It provides information on momentum, support and resistance. It is made up of 5 lines:
- Conversion Line (Tenkan-sen) : Represents support, resistance and reversals. Used to measure short term trends.
- Baseline (Kijun-sen) : Represents support, resistance and confirms trend changes. Allows you to evaluate the strength of medium term trends. Called the baseline because it lags the price.
- Leading Span A (Senkou A) : Used to identify future areas of support and resistance
- Leading Span B (Senkou B) : Other line used to identify suture support and resistance
- Lagging Span (Chikou) : Shows possible support and resistance. It is used to confirm signals obtained from other lines.
- Cloud (Kumo) : Space between Span A and B. Represents the divergence in price evolution.

A session is a day (in my case)
Formulas also from: https://www.investopedia.com/terms/i/ichimoku-cloud.asp

In [31]:
def add_ichimoku(df):
    hi26 = df['High'].rolling(window=26).max()
    hi52 = df['High'].rolling(window=52).max()
    
    lo26 = df['Low'].rolling(window=26).min()
    lo52 = df['Low'].rolling(window=52).min()
    
    # Conversion line = (Highest Value in period + Lowest value in period)/2 (9 sessions)
    df['conversion_line'] = 0.5 * (df['High'].rolling(window=9).max() + df['Low'].rolling(window=9).min())
    
    # Base line = (Highest Value in period + Lowest value in period)/2 (26 sessions)
    df['base_line'] = 0.5 * (hi26 + lo26)
    
    # Span A = (Conversion Value + Base Value)/2 (plot 26 Sessions into the future)
    df['span_A'] = (0.5 * (df['conversion_line'] + df['base_line'])).shift(26)
    
    # Span B = (Conversion Value + Base Value)/2 (plot 26 Sessions into the future)
    df['span_B'] = (0.5 * (hi52 + lo52)).shift(26)
        
    # Lagging Span = Price shifted back 26 periods
    df['lag_span'] = df['Close'].shift(-26)
    
    return df

In [ ]:
# TRY ON ONE STOCK (check all the functions are working)
# ticker = 'AMD'
# try:
#     print('Working on ', ticker)
#     test_df = get_stock_df_from_csv(ticker)
#     test_df = add_daily_return_to_df(test_df)
#     test_df = add_cumulative_return_to_df(test_df)
#     test_df = add_bollinger_bands(test_df)
#     test_df = add_ichimoku(test_df)
#     # test_df.to_csv(stocks_path + ticker + '.csv')
# except Exception as ex:
#     print(ex)
# test_df

# PLOTS
***

## Plot Bollinger bands

In [32]:
def plot_with_bollinger_bands(df, ticker):
    fig = go.Figure()
    
    candle = go.Candlestick(x=df.index, open=df['Open'],
                            high=df['High'], low=df['Low'],
                            close=df['Close'], name='Candlestick')
    
    upper_line = go.Scatter(x=df.index, y=df['upper_band'],
                            line=dict(color='rgba(250, 0, 0, 0.75)', width=1),
                            name='Upper Band')
    
    mid_line = go.Scatter(x=df.index, y=df['middle_band'],
                            line=dict(color='rgba(0, 250, 0, 0.75)', width=1),
                            name='Middle Band')
    
    lower_line = go.Scatter(x=df.index, y=df['lower_band'],
                            line=dict(color='rgba(0, 0, 250, 0.75)', width=1),
                            name='Lower Band')
    
    fig.add_trace(candle)
    fig.add_trace(upper_line)
    fig.add_trace(mid_line)
    fig.add_trace(lower_line)
    
    fig.update_xaxes(title="Date", rangeslider_visible=True)
    fig.update_yaxes(title="Price")
    
    fig.update_layout(title=ticker+" Bollinger bands",
                      height=1200, width=1800, showlegend=True)
    
    # plot(fig, auto_open=True)
    fig.show()

## Plot Ichimoku
The plot should display a buy decision (green) or not (red)

In [33]:
def get_fill_color(label):
    if label >= 1:
        return 'rgba(0, 250, 0, 0.4)'
    else:
        return 'rgba(250, 0, 0, 0.4)'
    
def plot_ichimoku(df, ticker, folder):
    candle = go.Candlestick(x=df.index, open=df['Open'],
                            high=df['High'], low=df['Low'],
                            close=df['Close'], name='Candlestick')
    
    df1 = df.copy()
    fig = go.Figure()
    df['label'] = np.where(df['span_A'] > df['span_B'], 1, 0) # condition for the color!!!
    
    df['group'] = df['label'].ne(df['label'].shift()).cumsum() # ne -> not equal
    # This last cumsum returns either a single True or False depending if 'label' changed value
    
    df = df.groupby('group') # group all those red or green zones
    
    dfs = [] # a list of dataframes
    for name, data in df:
        dfs.append(data)
        
    for df in dfs:
        fig.add_traces(go.Scatter(x=df.index, y=df.span_A,
                                  line=dict(color='rgba(0,0,0,0)')))
        fig.add_traces(go.Scatter(x=df.index, y=df.span_B,
                                  line=dict(color='rgba(0,0,0,0)'),
                                  fill='tonexty',
                                  fillcolor=get_fill_color(df['label'].iloc[0]))) # iloc -> finds where there are 0's
        
    baseline = go.Scatter(x=df1.index, y=df1['base_line'], 
                          line=dict(color='black', width=3), name='Baseline')
    
    conversion = go.Scatter(x=df1.index, y=df1['conversion_line'],
                            line=dict(color='blue', width=3), name='Conversion')
    
    lag = go.Scatter(x=df1.index, y=df1['lag_span'],
                            line=dict(color='#e09c00', width=2), name='Lag')
    
    span_a = go.Scatter(x=df1.index, y=df1['span_A'],
                            line=dict(color='red', width=2, dash='dot'), name='Span A')
    
    span_b = go.Scatter(x=df1.index, y=df1['span_B'],
                            line=dict(color='green', width=1, dash='dot'), name='Span B')
    
    fig.add_trace(candle)
    fig.add_trace(baseline)
    fig.add_trace(conversion)
    fig.add_trace(lag)
    fig.add_trace(span_a)
    fig.add_trace(span_b)
    
    fig.update_xaxes(title="Date", rangeslider_visible=True)
    fig.update_yaxes(title="Price")
    
    fig.update_layout(title = ticker, 
                      height=1200, width=1800, showlegend=True, 
                      plot_bgcolor = 'dimgray') # color from CSS colors list
    
    # Create a directory where to save the plots in case it doesn't exist
    folder_path = plots_path + folder + "\\"
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)
    
    # fig.show() # in case you just want to display
    plot(fig, filename= folder_path + ticker + '.html', auto_open=False) # in case you want to save

In [19]:
# Test the plot functions
# plot_with_bollinger_bands(test_df, ticker)
# plot_ichimoku(test_df, ticker, 'delete')  

## Add daily and cumulative gains to dataframes
To select the "best" later based in cumulative returns

TODO: (You could also select the best based in cloud width)

In [ ]:
# No reason to run this more than once
for t in tickers:
    try:
        print("Working on: ", t)
        new_df = get_stock_df_from_csv(t)
        new_df = add_daily_return_to_df(new_df)
        new_df = add_daily_return_to_df(new_df)
        new_df = add_cumulative_return_to_df(new_df)
        # new_df = add_ichimoku(new_df)
        new_df.to_csv(stocks_path + t + '.csv') 
    except Exception as ex:
        print(ex)

# SELECTION OF THE BEST PERFORMING STOCKS
Based on cumulative returns

## Organize stocks by their sectors

In [34]:
sec_df = pd.read_csv("D:\\webScrapping\\derekBanasTutorial\\big_stock_sectors.csv")

# don't do a loop for this, this way it is easy to separate them
indus_df = sec_df.loc[sec_df['Sector'] == "Industrial"]
health_df = sec_df.loc[sec_df['Sector'] == "Healthcare"]
it_df = sec_df.loc[sec_df['Sector'] == "Information Technology"]
comm_df = sec_df.loc[sec_df['Sector'] == "Communication"]
staple_df = sec_df.loc[sec_df['Sector'] == "Staples"]
discretion_df = sec_df.loc[sec_df['Sector'] == "Discretionary"]
utility_df = sec_df.loc[sec_df['Sector'] == "Utilities"]
financial_df = sec_df.loc[sec_df['Sector'] == "Financials"]
material_df = sec_df.loc[sec_df['Sector'] == "Materials"]
restate_df = sec_df.loc[sec_df['Sector'] == "Real Estate"]
energy_df = sec_df.loc[sec_df['Sector'] == "Energy"]

In [ ]:
# indus_df # for example

## Return df with cumulative return for all stocks in a sector

In [35]:
def get_cum_ret_for_sector(sector_df):
    ticks = []
    cum_rets=[]
    
    for index, row in sector_df.iterrows():
        df = get_stock_df_from_csv(row['Ticker'])
        
        if df is None:
            pass
        else:
            ticks.append(row['Ticker'])
            
            cum = df['cum_return'].iloc[-1] 
            cum_rets.append(cum)
            
    return pd.DataFrame({'Ticker': ticks, 'CUM_RET': cum_rets})

In [36]:
# These below print "File doesn't exist!" from the get_stock_df_from_csv function
# because there are tickers from big_stock_sectors.csv that are not in the 
# downloaded stock data 
industrial = get_cum_ret_for_sector(indus_df)
health_care = get_cum_ret_for_sector(health_df)
it = get_cum_ret_for_sector(it_df)
commun = get_cum_ret_for_sector(comm_df)
staples = get_cum_ret_for_sector(staple_df)
discretion = get_cum_ret_for_sector(discretion_df)
utility = get_cum_ret_for_sector(utility_df)
finance = get_cum_ret_for_sector(financial_df)
material = get_cum_ret_for_sector(material_df)
restate = get_cum_ret_for_sector(restate_df)
energy = get_cum_ret_for_sector(energy_df)

File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!
File doesn't exist!


## Look for the top 10 stocks of highest cumulative returs
Of each sector, over the whole downloaded period

In [37]:
top = 10

# TOP INDUSTRIAL
print("INDUSTRIAL")
top_ind = industrial.sort_values(by=['CUM_RET'], ascending=False).head(top)
print(top_ind)

# # TOP HEALTHCARE
print("HEALTHCARE")
top_health = health_care.sort_values(by=['CUM_RET'], ascending=False).head(top)
print(top_health)

# TOP IT
print("IT")
top_it = it.sort_values(by=['CUM_RET'], ascending=False).head(top)
print(top_it)

# TOP COMMUNICATION
print("COMMUNICATION")
top_com = commun.sort_values(by=['CUM_RET'], ascending=False).head(top)
print(top_com)

# TOP STAPLES
print("STAPLES")
top_staples = staples.sort_values(by=['CUM_RET'], ascending=False).head(top)
print(top_staples)

# TOP DISCRETION
print("DISCRETIONARY")
top_discretion = discretion.sort_values(by=['CUM_RET'], ascending=False).head(top)
print(top_discretion)

# TOP UTLITY
print("UTILITY")
top_utility = utility.sort_values(by=['CUM_RET'], ascending=False).head(top)
print(top_utility)

# TOP FINANCE
print("FINANCE")
top_finance = finance.sort_values(by=['CUM_RET'], ascending=False).head(top)
print(top_finance)

# TOP MATERIALS
print("FINANCE")
top_materials = material.sort_values(by=['CUM_RET'], ascending=False).head(top)
print(top_materials)

# TOP REAL STATE
print("REAL STATE")
top_restate = restate.sort_values(by=['CUM_RET'], ascending=False).head(top)
print(top_restate)

# TOP ENERGY
print("ENERGY")
top_energy = energy.sort_values(by=['CUM_RET'], ascending=False).head(top)
print(top_energy)

INDUSTRIAL
    Ticker    CUM_RET
249   PLUG  23.252688
27    AMRC   9.235643
71    CALX   7.006140
145   GNRC   6.918496
98    CWST   6.762491
325   VICR   6.396587
346   ZBRA   6.130492
222   NOVT   5.958170
240    PAR   5.775157
63    BLDR   5.763949
HEALTHCARE
    Ticker    CUM_RET
525   ZYXI  27.896037
62    ARWR  26.952127
325   MRTX  23.017142
363   OPRX  18.567100
116   CDNA  16.895832
97    BLFS  15.273743
173   DRNA  14.700001
203   FATE  13.889680
491   VCEL  12.596295
166   CYRX  11.655385
IT
    Ticker    CUM_RET
93    ENPH  98.795773
28    APPS  63.485508
283    TTD  25.027118
248   SEDG  19.653252
3     ACMR  14.053571
170    MDB  12.654818
99    ETSY  10.771475
309     WK   9.129921
19     AMD   8.935437
134   HUBS   8.610424
COMMUNICATION
   Ticker   CUM_RET
72   TTGT  9.663254
59   ROKU  6.806464
12   CDLX  5.019073
47    LYV  3.745030
80   ZNGA  3.600394
38   IRDM  3.465405
73   TTWO  3.229336
76     VG  3.081241
55    NYT  2.974738
5    BAND  2.968000
STAPLES
   Tick

## Plot the top ten of a sector (Function)

In [38]:
def plot_ichi_top_10(top_sector_df, sector):
    for t in top_sector_df['Ticker']:
        df_ind = get_stock_df_from_csv(t)
        df_ind = add_ichimoku(df_ind)
        plot_ichimoku(df_ind, t, sector) # this saves them in folders
        time.sleep(2) # in seconds
        # they have to be opened slowly, otherwise my machine does not have enough
        # time to write them to disk/ram and open them again in the browser

## Plot the top 10 for each sector
Remember the plot_ichimoku function can either plot or save the figure in a subfolder

In [39]:
plot_ichi_top_10(top_ind, 'industrial')
plot_ichi_top_10(top_health, 'healthcare')
plot_ichi_top_10(top_it, 'it')
plot_ichi_top_10(top_com, 'comms')
plot_ichi_top_10(top_staples, 'staples')
plot_ichi_top_10(top_discretion, 'discretionary')
plot_ichi_top_10(top_utility, 'utility')
plot_ichi_top_10(top_finance, 'finance')
plot_ichi_top_10(top_materials, 'materials')
plot_ichi_top_10(top_restate, 'realState')
plot_ichi_top_10(top_energy, 'energy')

## How to select a portfolio based on Ichimoku?
***
Take trades given the below conditions (in this order)
- FIRST
    - Price above cloud -> Take long trades
    - Price below cloud -> Take short trades
    - Price inside cloud -> Take no trades

- THEN
    - When conversion crosses the baseline upwards, it's a long trading signal
     -When conversion crosses the baseline downwards, it's a short trading signal

- FINALLY 
    - If you're taking long trades, check that the lagging signal is above price
    - Viceversa if you're taking short trades

- Exit signals
    - When the lagging signal touches price
    - When conversion crosses baseline again
    - When the cloud touches price

Stop loss can be set at the bottom/top of the cloud, when the position is taken

In [ ]:
portfolio = ['ZNGA', 'VG', 'ADM', 'MKC', 'HSY', 'XEL', 'EXC', 'VRS', 'HCC',
              'BCC', 'WHD', 'OAS', 'EGY']

In [ ]:
# Stocks to watch (e.g. price inside cloud, conversion crosses baseline upwards but price below cloud)
watch = ['CRC']